In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
! pip install --upgrade efficientnet-pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=ca4e352c96d6e52cd3058121b877a59ab30a8d50103207719020c57e2e586e6b
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch


In [2]:
import torch
import numpy as np
import torch
from torch.utils import data

# Import dataset related API
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import models
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# Import common neural network API in pytorch
import torch.nn as nn
import torch.nn.functional as F

# Import optimizer related API
import torch.optim as optim

# Import Efficient net model (pytorch implementation)
import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

# Fix the random seed of pytorch related function
torch.manual_seed(0)

# Fix the random seed of numpy related function
np.random.seed(0) 

# Check device, using gpu 0 if gpu exist else using cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Stage 1: Initial Training on Labeled Data Set
### No noise

In [0]:
transform = transforms.Compose([#transforms.ToPILImage()])
                               transforms.ToTensor(),
                               transforms.Normalize([.5,.5,.5],[.5,.5,.5])
                               ])

In [48]:
train_data = datasets.SVHN('drive/My Drive/',split='train',download=True,transform=transform)
test_data = datasets.SVHN('drive/My Drive/',split='test',download=True,transform=transform)

Using downloaded and verified file: drive/My Drive/train_32x32.mat
Using downloaded and verified file: drive/My Drive/test_32x32.mat


In [0]:
train_dataloader =  torch.utils.data.DataLoader(
        train_data, batch_size=1024
    )
test_dataloader = torch.utils.data.DataLoader(
        test_data, batch_size=1024
    )

None of the transforms in the first teacher model should have any noise.

In [91]:
#model = EfficientNet.from_name('efficientnet-b0')
model = EfficientNet.from_pretrained('efficientnet-b0')

Loaded pretrained weights for efficientnet-b0


In [0]:
num_ftrs = model._fc.in_features

model._fc = nn.Sequential(nn.Linear(num_ftrs, 10),
                         nn.Softmax(1))
model = model.to(device)

In [53]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003,momentum=.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#optimizer = optim.Adam(model.parameters())
#loss_func = nn.BCELoss()
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [0]:
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [58]:
print('Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy')
print('____________________________________________________________________')
q=0
for epoch in range(10):
    running_loss = 0.0
    running_corrects = 0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.to(device)
            labels = labels.to(device)


        # zero the parameter gradients: Clean the gradient caclulated in the previous iteration
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        #print(outputs.shape)
        #print(labels.shape)

        loss = criterion(outputs, labels.long())

        # Calculate gradient of matrix with requires_grad = True
        loss.backward()

        # Apply the gradient calculate from last step to the matrix
        optimizer.step()
        # Add 1 more iteration count to learning rate scheduler
        scheduler.step()

        # print statistics
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

    val_corrects = 0
    val_loss = 0
    model.eval()

    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data
            if torch.cuda.is_available():
                images = images.to(device)
                labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_loss += criterion(outputs, labels.long())
            num = torch.sum(preds == labels.data)
            val_corrects += num
    #print('Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy')

    print("  %2d   |    %.4f   |     %.4f       |   %.4f   |     %.4f    " 
          %(q,running_loss/len(train_data),int(running_corrects)/len(train_data),val_loss/len(test_data),int(val_corrects)/len(test_data)))

    q+=1




print('Finished Training')

Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy
____________________________________________________________________
   0   |    0.0016   |     0.8240       |   0.0017   |     0.8125    
   1   |    0.0016   |     0.8254       |   0.0017   |     0.8124    
   2   |    0.0016   |     0.8246       |   0.0017   |     0.8125    
   3   |    0.0016   |     0.8241       |   0.0017   |     0.8124    
   4   |    0.0016   |     0.8243       |   0.0017   |     0.8124    
   5   |    0.0016   |     0.8251       |   0.0017   |     0.8124    
   6   |    0.0016   |     0.8232       |   0.0017   |     0.8124    
   7   |    0.0016   |     0.8253       |   0.0017   |     0.8122    


KeyboardInterrupt: ignored

In [0]:
torch.save(model, 'drive/My Drive/model.pt')

# Stage 2: Predictions on unlabeled data

### In the case of SVHN this is the extra data

In [0]:
model = torch.load('drive/My Drive/model.pt')

In [60]:
extra_data = datasets.SVHN('drive/My Drive/',split='extra',download=True,transform=transform)

Using downloaded and verified file: drive/My Drive/extra_32x32.mat


In [0]:
extra_loader = torch.utils.data.DataLoader(
        extra_data, batch_size=248)

In [0]:
# Switch some layers (e.g., batch norm, dropout) to evaluation mode
model.eval()
# Turn off the autograd to save memory usage and speed up

prediction_list =[]
label_list = []
with torch.no_grad():
    for data in extra_loader:
        images, labels = data
        if torch.cuda.is_available():
            images = images.to(device)
            labels = labels.to(device)
        outputs = model(images)
        predicted = outputs.data
        prediction_list.append(predicted.cpu())
        label_list.append(labels.cpu())

In [0]:
p_list = torch.cat(prediction_list)
p_list = p_list.numpy()

### Select threshold

In [0]:
threshold = .99

In [0]:
labels = (p_list>threshold).astype(int)*np.array([10,1,2,3,4,5,6,7,8,9])
indexes = np.array(list(range(np.max(p_list.shape))))[list((np.sum(labels,axis=1)>.1))]
labels = np.argmax(labels[indexes],axis=1)
#set 10 back to 0
labels[labels==10] = 0

In [80]:
train_data = datasets.SVHN(
        root='drive/My Drive/', split='train',
        download=True, transform=transform,
    )

Using downloaded and verified file: drive/My Drive/train_32x32.mat


In [0]:
train_dataloader = torch.utils.data.DataLoader(
        train_data, batch_size=141,
    )

In [70]:
!pip install RandAugment

In [0]:
from randaugment import RandAugment, ImageNetPolicy

## Stage 2.1: Reload Unlabeled (Extra) Data and apply *RandAugment*

In [0]:
rand_transform = transforms.Compose([transforms.ToPILImage(),
                               RandAugment(),
                               transforms.ToTensor(),
                               transforms.Normalize([.5,.5,.5],[.5,.5,.5])
                               ])

In [83]:
aug_data = datasets.SVHN(
        root='drive/My Drive/', split='extra',
        download=True, transform=rand_transform,
    )
aug_dataloader = torch.utils.data.DataLoader(
        train_data, batch_size=(1024-141),
    )

Using downloaded and verified file: drive/My Drive/extra_32x32.mat


In [0]:
indices = np.array([False]*aug_data.data.shape[0])
indices[indexes] = True

In [85]:
#only need to run this cell when step 1 was skipped
test_data = datasets.SVHN('drive/My Drive/',split='test',download=True,transform=transform)
test_dataloader = torch.utils.data.DataLoader(
        test_data, batch_size=1024
    )

Using downloaded and verified file: drive/My Drive/test_32x32.mat


In [101]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=.9)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#optimizer = optim.Adam(model.parameters())
#loss_func = nn.BCELoss()
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [102]:
print('Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy')
print('____________________________________________________________________')
q=0
run_len = train_data.data.shape[0]+labels.shape[0]
test_len = test_data.data.shape[0]


for epoch in range(25):
    running_loss = 0.0
    running_corrects = 0
    model.train()
    inds = indices
    labs = torch.Tensor(labels)

    for i in enumerate(zip(train_dataloader,aug_dataloader)):
        

        # get the inputs; data is a list of [inputs, labels]
        bs = min(883,i[1][1][0].shape[0])
        aug = i[1][1][0][inds[:bs]]
        inputs = torch.cat([i[1][0][0],aug])

        l = torch.cat([i[1][0][1].float(),labs[:aug.shape[0]].float()])

        # randomize the batches
        ran = torch.randperm(l.shape[0])
        inputs = inputs[ran]
        l = l[ran]
        if torch.cuda.is_available():
            inputs = inputs.to(device)
            l = l.to(device)


        # zero the parameter gradients: Clean the gradient caclulated in the previous iteration
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        #print(outputs.shape)
        #print(labels.shape)

        loss = criterion(outputs, l.long())

        # Calculate gradient of matrix with requires_grad = True
        loss.backward()

        # Apply the gradient calculate from last step to the matrix
        optimizer.step()
        # Add 1 more iteration count to learning rate scheduler
        #scheduler.step()

        # print statistics
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == l.data)

    val_corrects = 0
    val_loss = 0
    model.eval()

    inds = inds[883:]

    with torch.no_grad():
        for data in test_dataloader:
            images, l = data
            if torch.cuda.is_available():
                images = images.to(device)
                l = l.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_loss += criterion(outputs, l.long())
            num = torch.sum(preds == l.data)
            val_corrects += num
    #print('Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy')

    print("  %2d   |    %.4f   |     %.4f       |   %.4f   |     %.4f    " 
          %(q,running_loss/run_len,int(running_corrects)/run_len,val_loss/test_len,int(val_corrects)/test_len))

    q+=1




print('Finished Training')

Epoch  | Train Loss  |   Train Accuracy |  Val Loss  |  Val Accuracy
____________________________________________________________________
   0   |    0.0003   |     0.0412       |   0.0022   |     0.2797    
   1   |    0.0003   |     0.0414       |   0.0022   |     0.2808    
   2   |    0.0003   |     0.0417       |   0.0022   |     0.2804    
   3   |    0.0003   |     0.0417       |   0.0022   |     0.2802    


KeyboardInterrupt: ignored